
# **Desafio** | Analista de Dados

---

Autora - [Tsad Martins](https://www.linkedin.com/in/tsadmborges/) <br>
Desenvolvido em Janeiro/2024 <br>

---


---

# 1\. Contexto

Avaliação e pré-aprovação de credito de clientes, Lojas XYZ, parceiras.<br>
A área de crédito definiu as seguintes regras:<br>
<ol type="1">
  
  <li>Clientes não podem ter tido renegociação;</li>
  <li>O atraso médio precisa ser menor que 10 dias e o atraso máximo histórico menor que 40 dias;</li>
  <li>Precisa estar com CPF regular na Receita Federal;</li>
  
</ol>


### **Definição do problema:**

Avaliar **quais clientes** podem ter um limite de crédito pré-aprovado e **qual o valor** desse limite.



# 2\. Pacotes e bibliotecas e Download dos dados


In [1]:
import pandas as pd #manipular tabela
import seaborn as sns # grafico
import numpy as np #operações numpy
import matplotlib.pyplot as plt #Grafico
#import sweetviz as sv #Resumo dos dados
import statistics as sts

In [2]:
#Antes de importar para o pyhton, foi tratado, no excel,  a coluna cod_cliente com a tabela dos clientes, alterando o formato.
# Calculado a idade do cliente pela data de nascimento
#Foi incluido as colunas (CPF status, genro, ID ponto, CD_ponto, idade)

In [3]:
#Importar csv´s
parceiro = pd.read_excel("/content/base_parceiro_LOJAS_XYZ 1 - alterada.xlsx")
parceiro.head(3)

,codigo,cidade,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade
0,100010,ITARARE,SP,2500.0,0,2.89,152.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,53
1,100017,ITARARE,SP,100.0,0,NaN,2.0,NaN,REGULAR,F,0,0,64
2,100025,ITARARE,SP,100.0,0,NaN,167.0,NaN,REGULAR,M,0,0,65


In [4]:
#Conferencia dos totais de linhas do csv, tabela clientes
parceiro.shape

(23161, 13)

In [5]:
# Ver campos nulos, campos com dados nulos (media_dias_atraso, max_dias_atraso_geral, numero_filial)
#Alguns tem nulos, pois as filiais 97 e 98 não tem detalhamento
parceiro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23161 entries, 0 to 23160
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   codigo                 23161 non-null  int64  
 1   cidade                 23161 non-null  object 
 2   estado                 23161 non-null  object 
 3   renda                  23161 non-null  float64
 4   qt_renegociacao        23161 non-null  int64  
 5   media_dias_atraso      10325 non-null  float64
 6   max_dias_atraso_geral  19453 non-null  float64
 7   numero_filial          22583 non-null  float64
 8   CPF_STATUS             23161 non-null  object 
 9   GENERO                 23161 non-null  object 
 10  ID_PONTO               23161 non-null  object 
 11  CD_DS_PONTO            23161 non-null  object 
 12  idade                  23161 non-null  int64  
dtypes: float64(4), int64(3), object(6)
memory usage: 2.3+ MB


# 3\. Exploração e manipulação dos dados

In [6]:
# TOTAL Genero
#Em percentual
analise = round (parceiro['GENERO'].value_counts(normalize = True) * 100, 3)
analise

F    60.718
M    27.982
I    11.299
Name: GENERO, dtype: float64

In [7]:
parceiro.head(3)

,codigo,cidade,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade
0,100010,ITARARE,SP,2500.0,0,2.89,152.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,53
1,100017,ITARARE,SP,100.0,0,NaN,2.0,NaN,REGULAR,F,0,0,64
2,100025,ITARARE,SP,100.0,0,NaN,167.0,NaN,REGULAR,M,0,0,65


In [8]:
#Linhas duplicadas
parceiro.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
23156    False
23157    False
23158    False
23159    False
23160    False
Length: 23161, dtype: bool

In [9]:
# exclur coluna cidade, não será utilizado na analise, melhora a visualização.
parceiro = parceiro.drop(columns = ['cidade'])

In [10]:
# Valores unicos da tabela clientes
parceiro.nunique()

codigo                   23161
estado                      10
renda                     1995
qt_renegociacao             50
media_dias_atraso         2444
max_dias_atraso_geral     1622
numero_filial               22
CPF_STATUS                   6
GENERO                       3
ID_PONTO                    16
CD_DS_PONTO                 16
idade                       81
dtype: int64

# 4\.  **Resolução do problema**

In [13]:
import plotly.express as px # para gerar boxplot com mais informações

In [11]:
#Dados estatisticos
# até 75% dos clientes, tem média de atraso de 9 dias
# E 75% tem renda até 2200,00
parceiro.describe()

,codigo,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,idade
count,23161.000000,23161.000000,23161.000000,10325.000000,19453.000000,22583.000000,23161.000000
mean,156673.526877,1953.658474,0.995121,11.052513,204.179972,14.086348,47.624023
std,35286.415578,4763.032407,3.583223,39.687997,649.903859,19.411191,15.463911
min,100010.000000,0.000000,0.000000,-126.000000,-31.000000,1.000000,20.000000
25%,127817.000000,1092.000000,0.000000,1.210000,10.000000,5.000000,35.000000
50%,156320.000000,1500.000000,0.000000,3.570000,31.000000,12.000000,46.000000
75%,185837.000000,2200.000000,0.000000,9.000000,78.000000,15.000000,59.000000
max,223910.000000,548389.000000,59.000000,723.000000,6160.000000,98.000000,103.000000


In [14]:
#Outliers renda
#upper fence = 3862 (limite superior) e  0 limite inferior
view2 = px.box(parceiro, y = 'renda')
view2.show()

In [15]:
#1536 clientes tem renda além do limite superior (Outliers)
outlier_sup = parceiro.loc[parceiro.renda >= 3862]
outlier_sup.shape

(1536, 12)

In [16]:
#330 clientes tem renda abaixo do limite inferior
outlier_inf = parceiro.loc[parceiro.renda <= 0]
outlier_inf.shape

(330, 12)

In [17]:
#Qual a representação dos outliers do total de clientes?
round ((len(outlier_sup)+ len(outlier_inf)) / len(parceiro), 2)

# 8% dos clientes tem renda de outliers, para deep diving, fazeria essa analise separada.

0.08

In [18]:
#representação dos estados
parceiro['estado'].value_counts()

SP    13563
PR     9558
SC       16
MG       13
RS        5
CE        2
PB        1
BA        1
RJ        1
ES        1
Name: estado, dtype: int64

In [19]:
#representação dos Genero
parceiro['GENERO'].value_counts()

F    14063
M     6481
I     2617
Name: GENERO, dtype: int64

In [20]:
#representação por ponto de venda
parceiro['CD_DS_PONTO'].value_counts()

                                   5662
8458-ITARARE 01 2967/02            2758
8468-JAGUARIAIVA 15 2967/12        2424
8465-GUAPIARA 12 2967/09           2135
8471- ITARARE 18 2967/15           1991
8466-SENGES 13 2967/10             1697
8462-ARAPOTI 07 2967/06            1214
8467-RIBEIRAO BRANCO 14 2967/11    1186
8470-PIRAI DO SUL 17 2967/14        854
8463-NOVA CAMPINA 09 2967/07        744
8459-BOM SUCESSO 04 2967/03         616
8460- APIAI 05 2967/04              613
8461-OURO VERDE 06 2967/05          486
8464-ITAPORANGA 11 2967/08          365
8472-WENCESLAU BRAZ 20 2967/16      352
0                                    64
Name: CD_DS_PONTO, dtype: int64

In [21]:
#representação dos Status dos CPF
cpf = parceiro['CPF_STATUS'].value_counts()
cpf

REGULAR                      22910
PENDENTE DE REGULARIZACAO      152
TITULAR FALECIDO                53
SUSPENSA                        38
0                                5
CANCELADA                        3
Name: CPF_STATUS, dtype: int64

In [22]:
22910/23162

0.9891201105258614

## 4.1\. Aplicando regras aos clientes

### Clientes não podem ter tido renegociação;

In [23]:
parceiro.head(3)


,codigo,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade
0,100010,SP,2500.0,0,2.89,152.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,53
1,100017,SP,100.0,0,NaN,2.0,NaN,REGULAR,F,0,0,64
2,100025,SP,100.0,0,NaN,167.0,NaN,REGULAR,M,0,0,65


In [24]:
#Filtrando os clientes com renegociação = 0
negociacao = parceiro.loc[parceiro['qt_renegociacao'] == 0]
negociacao

,codigo,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade
0,100010,SP,2500.0,0,2.89,152.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,53
1,100017,SP,100.0,0,NaN,2.0,NaN,REGULAR,F,0,0,64
2,100025,SP,100.0,0,NaN,167.0,NaN,REGULAR,M,0,0,65
3,100026,SP,0.0,0,NaN,497.0,NaN,REGULAR,I,0,0,47
4,100027,SP,0.0,0,NaN,3929.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,71
...,...,...,...,...,...,...,...,...,...,...,...,...
23156,223862,PR,4000.0,0,NaN,NaN,7.0,REGULAR,F,9039,8462-ARAPOTI 07 2967/06,29
23157,223870,PR,2418.0,0,NaN,NaN,17.0,REGULAR,I,9500,8470-PIRAI DO SUL 17 2967/14,35
23158,223878,PR,3500.0,0,NaN,NaN,20.0,REGULAR,M,5533,8472-WENCESLAU BRAZ 20 2967/16,29
23159,223888,PR,2000.0,0,NaN,NaN,7.0,REGULAR,I,9039,8462-ARAPOTI 07 2967/06,36


In [25]:
negociacao.shape

(19549, 12)

### O atraso médio precisa ser menor que 10 dias e o atraso máximo histórico menor que 40 dias;

In [26]:
#Partindo da base filtrada

atraso = negociacao[(negociacao['media_dias_atraso'] < 10) & (negociacao['max_dias_atraso_geral'] < 40) ]
atraso.head()

,codigo,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade
8,100059,SP,1500.0,0,0.59,27.0,18.0,REGULAR,M,9501,8471- ITARARE 18 2967/15,49
66,100336,SP,880.0,0,3.00,12.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,61
76,100380,SP,2500.0,0,7.80,20.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,55
77,100388,SP,5800.0,0,3.00,14.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,68
106,100503,SP,1400.0,0,0.33,4.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,83


In [27]:
# Só tem valores com 0 dias de renegociação (conferencia)
atraso.groupby('qt_renegociacao')['codigo'].sum().reset_index()

,qt_renegociacao,codigo
0,0,932777741


In [28]:
# Valores menores que 10
atraso.groupby('media_dias_atraso')['codigo'].sum().reset_index()

,media_dias_atraso,codigo
0,0.01,267365
1,0.02,687016
2,0.03,1129029
3,0.04,1162988
4,0.05,3563527
...,...,...
763,9.86,219041
764,9.88,387086
765,9.90,195025
766,9.92,418441


In [29]:
# Valores menores que 40
atraso.groupby('max_dias_atraso_geral')['codigo'].sum().reset_index()

,max_dias_atraso_geral,codigo
0,1.0,30414008
1,2.0,38817253
2,3.0,38650594
3,4.0,36751065
4,5.0,33623077
5,6.0,44305082
6,7.0,36334604
7,8.0,34982188
8,9.0,35095764
9,10.0,44177704


In [30]:
#Base nova filtrada
atraso.head(3)

,codigo,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade
8,100059,SP,1500.0,0,0.59,27.0,18.0,REGULAR,M,9501,8471- ITARARE 18 2967/15,49
66,100336,SP,880.0,0,3.00,12.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,61
76,100380,SP,2500.0,0,7.80,20.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,55


In [31]:
atraso.shape

(5299, 12)

### Precisa estar com CPF regular na Receita Federal.

In [32]:
atraso = atraso.loc[atraso['CPF_STATUS'] == 'REGULAR']
atraso

,codigo,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade
8,100059,SP,1500.0,0,0.59,27.0,18.0,REGULAR,M,9501,8471- ITARARE 18 2967/15,49
66,100336,SP,880.0,0,3.00,12.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,61
76,100380,SP,2500.0,0,7.80,20.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,55
77,100388,SP,5800.0,0,3.00,14.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,68
106,100503,SP,1400.0,0,0.33,4.0,18.0,REGULAR,F,9501,8471- ITARARE 18 2967/15,83
...,...,...,...,...,...,...,...,...,...,...,...,...
23085,222980,PR,1000.0,0,3.00,3.0,17.0,REGULAR,F,9500,8470-PIRAI DO SUL 17 2967/14,47
23092,223054,PR,2000.0,0,1.00,1.0,15.0,REGULAR,I,5078,8468-JAGUARIAIVA 15 2967/12,50
23093,223055,PR,1800.0,0,6.00,6.0,20.0,REGULAR,I,5533,8472-WENCESLAU BRAZ 20 2967/16,40
23094,223059,PR,2000.0,0,5.00,5.0,15.0,REGULAR,M,5078,8468-JAGUARIAIVA 15 2967/12,22


In [33]:
# Somente com status REGULAR
atraso.groupby('CPF_STATUS')['codigo'].count().reset_index()

,CPF_STATUS,codigo
0,REGULAR,5254


## 4.2\. Calculo do limite pré-aprovado

### O cálculo de limite é obtido a partir de um percentual da Renda, conforme regras a seguir:
• Idade de 18 a 30 anos: 60% da Renda<br>
•
Idade de 31 a 40 anos: 90% da Renda<br>
•
Para idade de 41 a 60 anos: 120% da Renda<br>
•
Para idade de 60 a 75 anos e sexo feminino ou de SP: 180% da Renda<br>
•
Para idade de 60 a 75 anos e sexo masculino ou ser de PR: 150% da Renda<br>
•
Demais: 60% da Renda<br>

Além disso o limite atribuído deve ser um valor inteiro entre 500 e 3.000.

In [34]:
atraso.head(3)

,codigo,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade
8,100059,SP,1500.0,0,0.59,27.0,18.0,REGULAR,M,9501,8471- ITARARE 18 2967/15,49
66,100336,SP,880.0,0,3.00,12.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,61
76,100380,SP,2500.0,0,7.80,20.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,55


In [35]:
# Função para calcular o 'limite_pre'
def limite_pre(row):
    if 18 <= row['idade'] <= 30:
        limite = 0.6 * row['renda']
    elif 31 <= row['idade'] <= 40:
        limite = 0.9 * row['renda']
    elif 41 <= row['idade'] <= 60:
        limite = 1.2 * row['renda']
    elif 60 <= row['idade'] <= 75 and (row['GENERO'] == 'F' or row['estado'] == 'SP'):
        limite = 1.8 * row['renda']
    elif 60 <= row['idade'] <= 75 and (row['GENERO'] == 'M' or row['estado'] == 'PR'):
        limite = 1.5 * row['renda']
    else:
        limite = 0.6 * row['renda']

    # Garantir que o limite esteja no intervalo [500, 3000]
    return max(500, min(3000, int(limite)))

# Aplicar a função à nova coluna 'limite_pre'
atraso['pre_aprovado'] = atraso.apply(limite_pre, axis=1)


In [36]:
atraso.head(3)

,codigo,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,numero_filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade,pre_aprovado
8,100059,SP,1500.0,0,0.59,27.0,18.0,REGULAR,M,9501,8471- ITARARE 18 2967/15,49,1800
66,100336,SP,880.0,0,3.00,12.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,61,1584
76,100380,SP,2500.0,0,7.80,20.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,55,3000


In [37]:
#14 pontos, estão na tabela de pontos de venda

atraso['ID_PONTO'].unique()

array([9501, 9038, 9498, 9040, 5075, 9497, 5074, 9496, ' ', 5078, 5077,
       9500, 9039, 5533, 5076], dtype=object)

In [38]:
atraso.dtypes

codigo                     int64
estado                    object
renda                    float64
qt_renegociacao            int64
media_dias_atraso        float64
max_dias_atraso_geral    float64
numero_filial            float64
CPF_STATUS                object
GENERO                    object
ID_PONTO                  object
CD_DS_PONTO               object
idade                      int64
pre_aprovado               int64
dtype: object

### Selecionar clientes com o valor e ID_ponto
<br>

Depois de definido o público é necessário imputar estes clientes pré-aprovados no sistema da Viacerta para liberar os limites para os pontos de venda fazerem a oferta. <br>

Para isso é necessário buscar o ID_PONTO no arquivo de pontos de venda Viacerta correspondente a filial recebida no arquivo do parceiro. Por exemplo, a filial número 18 deve coincidir com '8471- ITARARE 18 2967/15'.

In [39]:
#Renomeando algumas colunas, para melhor entendimento

final = atraso.rename(columns={'codigo': 'codigo_cliente', 'numero_filial': 'filial'})
final.head(3)

,codigo_cliente,estado,renda,qt_renegociacao,media_dias_atraso,max_dias_atraso_geral,filial,CPF_STATUS,GENERO,ID_PONTO,CD_DS_PONTO,idade,pre_aprovado
8,100059,SP,1500.0,0,0.59,27.0,18.0,REGULAR,M,9501,8471- ITARARE 18 2967/15,49,1800
66,100336,SP,880.0,0,3.00,12.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,61,1584
76,100380,SP,2500.0,0,7.80,20.0,1.0,REGULAR,F,9038,8458-ITARARE 01 2967/02,55,3000


In [40]:
#selecionando somente as colunas necessárias para gerar o arquivo
#Organizar na ordem que for melhor para leitura, do parceiro
colunas_filtro = ['codigo_cliente', 'ID_PONTO', 'CD_DS_PONTO', 'filial', 'pre_aprovado', 'renda', 'CPF_STATUS']

clientes_pre_aprovados = final[colunas_filtro]
clientes_pre_aprovados

,codigo_cliente,ID_PONTO,CD_DS_PONTO,filial,pre_aprovado,renda,CPF_STATUS
8,100059,9501,8471- ITARARE 18 2967/15,18.0,1800,1500.0,REGULAR
66,100336,9038,8458-ITARARE 01 2967/02,1.0,1584,880.0,REGULAR
76,100380,9038,8458-ITARARE 01 2967/02,1.0,3000,2500.0,REGULAR
77,100388,9501,8471- ITARARE 18 2967/15,18.0,3000,5800.0,REGULAR
106,100503,9501,8471- ITARARE 18 2967/15,18.0,840,1400.0,REGULAR
...,...,...,...,...,...,...,...
23085,222980,9500,8470-PIRAI DO SUL 17 2967/14,17.0,1200,1000.0,REGULAR
23092,223054,5078,8468-JAGUARIAIVA 15 2967/12,15.0,2400,2000.0,REGULAR
23093,223055,5533,8472-WENCESLAU BRAZ 20 2967/16,20.0,1620,1800.0,REGULAR
23094,223059,5078,8468-JAGUARIAIVA 15 2967/12,15.0,1200,2000.0,REGULAR


In [41]:
4829/23162

0.20848804075641136

### O valor que o parceiro precisa ter disponibilidade, é de:

In [44]:
soma  = clientes_pre_aprovados['pre_aprovado'].sum()

print (" O valor para contemplar todos os emprestimos é de: R$" , soma)

 O valor para contemplar todos os emprestimos é de: R$ 9894014


#### Extrair dataframe para Excel e enviar para o parceiro.

In [ ]:
clientes_pre_aprovados.to_excel('clientes_pre_aprovados.xlsx', encoding='latin', index=False)

## Extra: Perguntas para fazer na apresentação/dashboard.



In [ ]:
regular.groupby('CPF_STATUS')['codigo'].count()

#### Percentual de clientes com CPF regular?

In [ ]:
#Percentual de clientes com CPF regular?
regular.head(3)


In [ ]:
round (len(regular) / len(parceiro) * 100, 2)

#### Percentual de clientes que nunca fizeram negociação?

In [ ]:

round (len(negociacao) / len(parceiro) * 100, 2)


#### Perc. de clientes que serão levados para pre - aprovação

In [ ]:
round (len(clientes_pre_aprovados) / len(parceiro) * 100, 2)

In [ ]:
clientes_pre_aprovados.head(3)

#### Próximos passos se for fazer associações e correlações das variáveis

In [ ]:
from scipy.stats import chi2_contingency #Tabela para calcular o Qui-quadrado

In [ ]:
!pip install bioinfokit #outro pacote para Qui-quadrado

In [ ]:
import scipy.stats as stats # Normalidade
import statistics as sts

In [ ]:
#Existe diferença em renda de homens e mulheres?

renda = pd.crosstab(index = final['GENERO'], columns = final['renda'])
renda

In [ ]:
chi2_contingency(renda)

In [ ]:
#Existe diferença no limite pre-aprovado entre homens e mulheres?

aprovado = pd.crosstab(index = final['GENERO'], columns = final['pre_aprovado'])
aprovado

In [ ]:
chi2_contingency(aprovado)

In [ ]:
# Média de limite aprovado
media_c = final['pre_aprovado'].mean()
media_c = round(media_c, 2)
media_c

In [ ]:
#Os valores não seguem uma normalidade, por ter valores muito alto nos extremos, mas até 75% dos valores movimentados estão em -0,43 e 0,87 centavos.
final.describe()


In [ ]:
#Das 14 filiais, qual filial teve mais limites aprovados?
# round (parceiro['ID_PONTO'].value_counts(normalize = True) * 100, 1)

parceiro['ID_PONTO'].value_counts()

In [ ]:
final['ID_PONTO'].value_counts()

In [ ]:
#Das 14 filiais, qual filial teve mais limites aprovados?
round (final['ID_PONTO'].value_counts(normalize = True) * 100, 1)


**Dashboard**: Fazer as visualizações no power BI